# FILTER METADATA

In [13]:
import pandas as pd
def read_tsv(tsv_file):
    df = pd.read_csv(tsv_file, sep='\t')
    return df

In [ ]:
# takes around 1.5 minutes to read the file
metadata = read_tsv('metadata-0615/metadata.tsv')

In [ ]:
metadata.head()
# print colnames
print(metadata.columns)

In [ ]:
# keep rows with complete and high coverage genome
print(metadata.shape)
metadata_filtered = metadata[(metadata['Is complete?'] == True) & (metadata['Is high coverage?'] == True)]
print(metadata_filtered.shape)

In [ ]:
important_columns = ['Virus name', 'Accession ID', 'Variant', 'Pango lineage', 'Clade', 'Collection date', 'Submission date', 'Location', 'Host']
metadata_filtered = metadata_filtered[important_columns].reset_index(drop=True)
metadata_filtered.head()

In [ ]:
# write the filtered metadata to a tsv file
metadata_filtered.to_csv('metadata-0615/metadata_filtered.tsv', sep='\t', index=False)

# FILTER SEQUENCES

In [83]:
selected_variant = 'Eris' # 'New_Omicron' or 'Eris'

In [84]:
import pandas as pd
from Bio import SeqIO

def read_fasta(fasta_file):
    fasta_sequences = SeqIO.parse(open(fasta_file, encoding="iso-8859-1"), 'fasta')
    return fasta_sequences

In [85]:
df_metadata = pd.read_csv("metadata-0615/metadata_filtered.tsv", sep="\t")

In [86]:
omic_acc, eris_acc = '', ''
# print variant names including Eris or Omicron words
unique_variants = df_metadata['Variant'].unique().tolist()
# print names including Eris or Omicron
for variant in unique_variants:
    acc = str(variant)
    if 'B.1.1.529' in acc:
        omic_acc = acc
    if 'EG.5' in acc:
        eris_acc = acc

print(omic_acc)
print(eris_acc)

Former VOC Omicron GRA (B.1.1.529+BA.*) first detected in Botswana/Hong Kong/South Africa
VOI GRA (EG.5+EG.5.*) first detected in Indonesia/France


In [87]:
if selected_variant == 'New_Omicron':
    selected_acc = omic_acc
elif selected_variant == 'Eris':
    selected_acc =  eris_acc

# get accession ids for a specific variant
df_selected = df_metadata[df_metadata['Variant'] == selected_acc]
print(len(df_selected))

6363


In [88]:
# filter out rows with missing accession ids
df_selected = df_selected[df_selected['Accession ID'].notnull()]
print(len(df_selected))

6363


In [89]:
# sort by date and print the last 10 dates
df_selected = df_selected.sort_values('Submission date')
# keep rows only with collection date is newer than 2023-05-20
if selected_variant == 'New_Omicron':
    df_selected_date = df_selected[df_selected['Submission date'] > '2023-06-01']
elif selected_variant == 'Eris':
    df_selected_date = df_selected[df_selected['Submission date'] > '2023-06-01']
print(len(df_selected_date))

6349


In [90]:
# keep only the collection data is in yyyy-mm-dd format
#df_selected_date = df_selected_date[df_selected_date['Collection date'].str.match(r'\d{4}-\d{2}-\d{2}')]
#df_selected_date = df_selected_date.reset_index(drop=True)

In [91]:
fseq_acc_filtered = read_fasta('spikeprot0508/spikeprot0508_acc_filtered.fasta') # created by filter_fseq.py

In [92]:
# Variant selection (Takes around 1 minute)

from tqdm import tqdm

# Convert the list to a set for faster lookup
selected_accs_set = set(df_selected_date['Accession ID'].tolist())

# Filter sequences using list comprehension
fseq_selected = [record for record in tqdm(fseq_acc_filtered) if record.id in selected_accs_set]

print(len(fseq_selected))

42515it [00:00, 213205.43it/s]

11445409it [00:54, 211152.20it/s]

4638


In [95]:
print("initial:      ", len(fseq_selected))
# remove sequences shorter than 1235
fseq_selected_hq = [record for record in fseq_selected if len(record.seq) > 1235]
print("length filter:", len(fseq_selected_hq))
# remove sequences with more than 13 Xs
fseq_selected_hq = [record for record in fseq_selected_hq if str(record.seq).count('X') < 13]
print("X filter:     ", len(fseq_selected_hq))

initial:       4638
length filter: 4638
X filter:      4638


In [96]:
# write the spike protein sequences for a specific variant to a fasta file
with open(f'spikeprot0508/spikeprot0508_acc_filtered_{selected_variant}.fasta', 'w') as f:
    for record in fseq_selected_hq:
        f.write('>' + record.id + '\n')
        f.write(str(record.seq) + '\n')

In [97]:
# get unique sequences for a specific variant
def get_unique_sequences(fasta_file):
    unique_sequences = []
    unique_ids = []
    for record in SeqIO.parse(open(fasta_file, encoding="iso-8859-1"), 'fasta'):
        if str(record.seq) not in unique_sequences:
            unique_sequences.append(str(record.seq))
            unique_ids.append(record.id)
    return unique_sequences, unique_ids

unique_sequences, unique_ids = get_unique_sequences(f'spikeprot0508/spikeprot0508_acc_filtered_{selected_variant}.fasta')

print(len(unique_sequences))

# write the unique sequences to a fasta file
with open(f'spikeprot0508/spikeprot0508_acc_filtered_{selected_variant}_unique.fasta', 'w') as f:
    for i, seq in enumerate(unique_sequences):
        f.write('>' + unique_ids[i] + '\n')
        f.write(seq + '\n')


2078


In [ ]:
# write the unique sequences to a fasta file
with open(f'spikeprot0508/spikeprot0508_acc_filtered_{selected_variant}_unique.fasta', 'w') as f:
    for i, seq in enumerate(unique_sequences):
        f.write('>' + unique_ids[i] + '\n')
        f.write(seq + '\n')

In [66]:
# write to a csv file
import pandas as pd
from Bio import SeqIO

def write_to_csv(fasta_file, csv_file):
    with open(csv_file, 'w') as f:
        f.write('accession_id,sequence\n')
        for record in SeqIO.parse(open(fasta_file, encoding="iso-8859-1"), 'fasta'):
            f.write(record.id + ',' + str(record.seq) + '\n')

write_to_csv(f'spikeprot0508/spikeprot0508_acc_filtered_{selected_variant}_unique.fasta',
             f'spikeprot0508/unique_{selected_variant}_{len(unique_sequences)}.csv')

# read the csv file
unique_selected_sequences = pd.read_csv(f'spikeprot0508/unique_{selected_variant}_{len(unique_sequences)}.csv')
unique_selected_sequences

,accession_id,sequence
0,EPI_ISL_17740503,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
1,EPI_ISL_17741975,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
2,EPI_ISL_17742593,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
3,EPI_ISL_17742616,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
4,EPI_ISL_17742596,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
...,...,...
2000,EPI_ISL_19084464,MFVFLVLLPLVSSQCVNLITTTQXXXXYTNSFTRGVYYPDKVFRSS...
2001,EPI_ISL_19084596,MFVFLVLLPLVSSQCVNLITRTQLSPAYTNSFTRGVYYPDKVFRSS...
2002,EPI_ISL_18229220,MFVFLVLLPLVSSQCVNLITRTQLSPAYTNSFTRGVYYPDKVFRSS...
2003,EPI_ISL_19086907,MFVFLVLLPLVSSQCVMPLFNLITTTQSYTNSFTRGVYYPDKVFRS...
